In [11]:
import numpy as np
from quri_parts.circuit import H, CZ
from sklearn.datasets import load_iris, load_digits
from sklearn.preprocessing import MinMaxScaler
from circuit import LearningCircuit
from numpy.typing import NDArray
from quri_parts.qulacs.overlap_estimator import create_qulacs_vector_overlap_estimator
from quri_parts.core.state import quantum_state, GeneralCircuitQuantumState


def create_quantum_circuit():
    qc = LearningCircuit(n_qubits)

    def preprocess_x(x: NDArray[np.float64], index: int) -> float:
        xa = x[index % len(x)]
        return min(1, max(-1, xa))

    for i in range(n_qubits):
        qc.add_gate(H(i))
    for d in range(depth):
        for i in range(n_qubits):
            qc.add_input_RY_gate(i, lambda x, i=i: preprocess_x(x, i))
        for i in range(n_qubits):
            qc.add_input_RX_gate(i, lambda x, i=i: preprocess_x(x, i))
        if d < depth - 1:
            for i in range(n_qubits):
                qc.add_gate(CZ(i, (i + 1) % n_qubits))
    return qc


X_train, y_train = load_iris(return_X_y=True)
# X_train = X_train / 16.
# X_train = X_train[:200]
# y_train = y_train[:200] #500 => fidelity 81.6[s]
n_qubits = 12
depth = 1
scaler = MinMaxScaler((0, np.pi / 2))
n_qubits = 12
depth = 1

X_train = scaler.fit_transform(X_train)

pqc = create_quantum_circuit()
x_0 = pqc.bind_input_and_parameters(X_train[0], [])
x_1 = pqc.bind_input_and_parameters(X_train[15], [])
estimator = create_qulacs_vector_overlap_estimator()
print(
    estimator(
        quantum_state(n_qubits=n_qubits, circuit=x_0), quantum_state(n_qubits=n_qubits, circuit=x_1)
    )
)

KeyboardInterrupt: 

In [ ]:
sq_distance = np.zeros((len(X_train), len(X_train)))
for i in range(len(X_train)):
    for j in range(i + 1, len(X_train)):
        inner_prod = estimator(
            quantum_state(n_qubits=n_qubits, circuit=pqc.bind_input_and_parameters(X_train[i], [])),
            quantum_state(n_qubits=n_qubits, circuit=pqc.bind_input_and_parameters(X_train[j], [])),
        )
        sq_distance[i][j] = 1 - inner_prod[0].real
        sq_distance[j][i] = sq_distance[i][j]
print(sq_distance)

[[0.         0.12065636 0.06844121 ... 0.84985485 0.80410116 0.80592477]
 [0.12065636 0.         0.02461169 ... 0.84485567 0.83054096 0.7926377 ]
 [0.06844121 0.02461169 0.         ... 0.86451119 0.83848435 0.81397022]
 ...
 [0.84985485 0.84485567 0.86451119 ... 0.         0.09282863 0.0760118 ]
 [0.80410116 0.83054096 0.83848435 ... 0.09282863 0.         0.09130359]
 [0.80592477 0.7926377  0.81397022 ... 0.0760118  0.09130359 0.        ]]


In [12]:
import numpy as np
import torch

_A = np.arange(12, dtype=np.float64).reshape(12, -1)
print(_A)
A = torch.tensor(_A, requires_grad=True, dtype=torch.float32)
cdist = torch.cdist(A, A)
q_tmp = 1 / (1 + cdist)
q_sum = torch.sum(q_tmp)
print(q_sum)
q_sum.backward()
print(A.grad)
print(q_sum.grad)

[[ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]]
tensor(44.6835, grad_fn=<SumBackward0>)
tensor([[ 1.1300],
        [ 0.6161],
        [ 0.3773],
        [ 0.2323],
        [ 0.1276],
        [ 0.0408],
        [-0.0408],
        [-0.1276],
        [-0.2323],
        [-0.3773],
        [-0.6161],
        [-1.1300]])
None


C:\Users\kazum\AppData\Local\Temp\ipykernel_17212\3878909853.py:13: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(q_sum.grad)


In [13]:
a = q_tmp.detach().numpy()
a = -np.power(a, 2)
# print(a)
from scipy.spatial import distance

y = _A
print(y.shape)
print(a[0, :].reshape(-1, 1).shape)
yy = distance.cdist(y, y, "minkowski", p=1)
print(yy[0, :])
y0_1 = a[0, :].reshape(-1, 1).T @ (2 * yy[0, :].reshape(-1, 1))
y0_2 = a[:, 0].reshape(-1, 1).T @ (-2 * yy[:, 0].reshape(-1, 1))
print(yy)
print(y0_1, y0_2)

(12, 1)
(12, 1)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [ 1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 2.  1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 3.  2.  1.  0.  1.  2.  3.  4.  5.  6.  7.  8.]
 [ 4.  3.  2.  1.  0.  1.  2.  3.  4.  5.  6.  7.]
 [ 5.  4.  3.  2.  1.  0.  1.  2.  3.  4.  5.  6.]
 [ 6.  5.  4.  3.  2.  1.  0.  1.  2.  3.  4.  5.]
 [ 7.  6.  5.  4.  3.  2.  1.  0.  1.  2.  3.  4.]
 [ 8.  7.  6.  5.  4.  3.  2.  1.  0.  1.  2.  3.]
 [ 9.  8.  7.  6.  5.  4.  3.  2.  1.  0.  1.  2.]
 [10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.  1.]
 [11. 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.  0.]]
[[-3.07646821]] [[3.07646821]]


In [22]:
import autograd.numpy as np
from autograd import grad
A = np.array([
    [1, 2, 3],
    [2, 4, 5],
    [3, 5, 1]
])

a = np.array([[1],[2],[3]],dtype=np.float64)
def cdist(a,b):
    return np.sqrt(np.sum((a[:, None] - b)**2, axis=2)) + 1e-6

def l(A):
    # A = cdist(a,a)
    q_tmp = 1 / (1 + A)
    q_sum = np.sum(q_tmp)
    return q_sum
grad_l = grad(l)
print(cdist(a,a))
print(torch.cdist(torch.tensor(a), torch.tensor(a)))
print(grad_l(A))

[[1.000000e-06 1.000001e+00 2.000001e+00]
 [1.000001e+00 1.000000e-06 1.000001e+00]
 [2.000001e+00 1.000001e+00 1.000000e-06]]
tensor([[0., 1., 2.],
        [1., 0., 1.],
        [2., 1., 0.]], dtype=torch.float64)
[[-0.25       -0.11111111 -0.0625    ]
 [-0.11111111 -0.04       -0.02777778]
 [-0.0625     -0.02777778 -0.25      ]]
